In [36]:
import tensorflow as tf
import matplotlib.pyplot as plt
import heapq
import cv2
import pandas as pd
from tensorflow import keras
import numpy as np
import os
import json
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# 0. Data Preprocessing

In [2]:
#Feature Normalization:
scaler = StandardScaler()

# 1. One-Feature-Model:   

Regard application swichting as the only feature, 
e.g. instagram --> whatsapp 
instagram as X_data
whatsapp as y_label


In [3]:
account_path = r"D:\Dokuments\Project_of_HCI\emva-dataset\EMVA_from_gDrive\ba704297-b90c-4761-b706-728379fa9815"
input_raw = []
label_raw = []
temp_switchingTime = []
temp_switchingTimeDate = []
duration = []

for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))

    list_session = []
    for file in os.listdir(recording_path):
        if str(file)[:7] == "Session":
            list_session.append(str(file))

    #if list_session==[]:
    #    continue    
    
    with open (os.path.join(recording_path, "application_data.txt")) as application_file:
        application_data = json.load(application_file)["application_data"]
        application_index = 0
        while application_index < len(application_data):
            if (application_data[application_index]["name"] != "com.miui.home" 
            and application_data[application_index]["name"] != "ch.ethz.inf.vs.staals.visualattentionrecorder"):
                num = 1
                while application_index+num < len(application_data):
                    if (application_data[application_index+num]["name"] != "com.miui.home" 
                    and application_data[application_index+num]["name"] != "ch.ethz.inf.vs.staals.visualattentionrecorder"
                    and application_data[application_index+num]["name"] != application_data[application_index]["name"]):
                        input_raw.append (application_data[application_index]["name"])
                        label_raw.append (application_data[application_index+num]["name"])
                        temp_switchingTime.append (application_data[application_index+1]["time"])
                        temp_switchingTimeDate.append (application_data[application_index+1]["timeDate"])
                        duration.append (application_data[application_index+num]["time"] - application_data[application_index]["time"])
                        break
                    else:
                        num += 1
            
            application_index += 1

print("INPUT DATA:")
print(len(input_raw))
print(input_raw)
print("----------------------------------------------------------------")
print("LABEL DATA:")
print(len(label_raw))
print(label_raw)
print(temp_switchingTime)
feature_switchingTime = temp_switchingTime

INPUT DATA:
665
['com.android.settings', 'com.android.calendar', 'com.android.calendar', 'com.google.android.youtube', 'com.android.chrome', 'com.google.android.youtube', 'com.android.email', 'com.google.android.youtube', 'com.facebook.katana', 'com.espn.score_center', 'com.espn.score_center', 'com.instagram.android', 'com.google.android.youtube', 'com.google.android.youtube', 'com.whatsapp', 'com.google.android.youtube', 'com.whatsapp', 'com.google.android.youtube', 'com.android.settings', 'com.google.android.youtube', 'de.motain.iliga', 'com.whatsapp', 'com.google.android.youtube', 'com.whatsapp', 'com.whatsapp', 'com.android.chrome', 'com.whatsapp', 'com.google.android.youtube', 'com.google.android.youtube', 'com.android.chrome', 'com.android.chrome', 'com.popularapp.thirtydayfitnesschallenge', 'com.whatsapp', 'com.android.email', 'com.whatsapp', 'com.facebook.katana', 'com.google.android.apps.maps', 'com.whatsapp', 'com.android.deskclock', 'com.whatsapp', 'com.google.android.apps.m

In [4]:
app = []
for i in range(len(input_raw)):
    signal = 1
    for j in range(len(app)):
        if app[j]==input_raw[i]:
            signal = 0
            break 
    if signal!=0:
        app.append(input_raw[i])
for i in range(len(label_raw)):
    signal = 1
    for j in range(len(app)):
        if app[j]==label_raw[i]:
            signal = 0
            break 
    if signal!=0:
        app.append(label_raw[i])
input = []
for element in input_raw:
    input.append(app.index(element))
print(input)
label = []
for element in label_raw:
    label.append(app.index(element))
print(label)

[0, 1, 1, 2, 3, 2, 4, 2, 5, 6, 6, 7, 2, 2, 8, 2, 8, 2, 0, 2, 9, 8, 2, 8, 8, 3, 8, 2, 2, 3, 3, 10, 8, 4, 8, 5, 11, 8, 12, 8, 11, 8, 2, 8, 8, 4, 4, 2, 8, 8, 2, 2, 2, 8, 8, 13, 2, 13, 8, 4, 2, 4, 2, 2, 8, 2, 2, 2, 4, 2, 4, 14, 4, 15, 2, 2, 16, 13, 2, 13, 16, 3, 2, 13, 2, 2, 2, 8, 2, 8, 2, 2, 3, 13, 2, 8, 13, 2, 8, 13, 2, 8, 13, 2, 8, 13, 2, 8, 13, 8, 13, 2, 8, 8, 8, 17, 17, 3, 3, 3, 3, 3, 2, 2, 2, 2, 5, 2, 2, 7, 4, 7, 2, 3, 2, 2, 3, 2, 13, 3, 18, 2, 2, 3, 8, 8, 4, 8, 16, 15, 16, 3, 8, 3, 2, 13, 15, 2, 8, 8, 2, 2, 2, 8, 2, 8, 8, 8, 8, 8, 19, 2, 18, 2, 8, 2, 8, 2, 13, 8, 13, 2, 3, 13, 2, 2, 2, 8, 5, 8, 2, 2, 2, 2, 6, 2, 2, 20, 13, 2, 4, 1, 13, 4, 8, 4, 2, 4, 2, 12, 2, 2, 15, 21, 11, 11, 3, 2, 4, 2, 2, 2, 3, 2, 8, 2, 8, 8, 16, 3, 16, 3, 8, 5, 2, 3, 16, 15, 22, 15, 16, 3, 16, 2, 15, 8, 2, 13, 8, 2, 8, 2, 2, 8, 6, 7, 7, 2, 2, 2, 2, 3, 13, 2, 15, 13, 2, 2, 15, 13, 15, 13, 2, 2, 11, 2, 2, 23, 24, 23, 13, 2, 8, 7, 25, 26, 27, 28, 26, 28, 26, 28, 26, 28, 26, 28, 26, 28, 26, 28, 26, 2, 2, 12, 2, 2,

In [5]:
feature_app = np.array(input)
np_label = np.array(label)
feature_app = feature_app.reshape(len(input_raw),1)
len(app)

41

In [6]:
feature_app.shape

(665, 1)

# from app(1 feature) to app

In [7]:
X = scaler.fit_transform(feature_app)
y = np_label
y.shape

(665,)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## kNN Model:

In [9]:
kNN = KNeighborsClassifier(n_neighbors=7, weights='distance').fit(X_train,y_train) #probabilistic 
model_1_kNN = kNN.score(X_test, y_test)
print(model_1_kNN)

0.3413173652694611


# Random Forest Model:

In [10]:
rf = RandomForestClassifier(n_estimators=30)
rf.fit(X_train,y_train)
model_1_rf = rf.score(X_test, y_test)
print(model_1_rf)

0.33532934131736525


# 2. Model plus video features:

Video features are extracted from .csv file
Still with 3 different time windows

In [52]:
def extractFeatures(df_temp, switchingTime, sessionStartTime, timeWindow):
    df = df_temp.iloc[:,2:]
    index_timestamp_list=[]
    for index,timestamp in df["timestamp"].items():
        if timestamp >= (switchingTime-sessionStartTime)/1_000_000_000 - timeWindow and timestamp <= (switchingTime-sessionStartTime)/1_000_000_000:
            index_timestamp_list.append(index)

    df_timestamp_window = pd.DataFrame(columns=df.columns.values.tolist())
    for i in index_timestamp_list:
        df_timestamp_window=df_timestamp_window.append(df.iloc[i],ignore_index=True)

    index_confidence_list=[]
    for index,confidence in df_timestamp_window["confidence"].items():#use confidence or we could use "sucess==1" to check the validity
        if confidence > 0.8:
            index_confidence_list.append(index)

    df_window = pd.DataFrame(columns=df.columns.values.tolist())#contains the valid and high confidence features for all frames during the time window
    for i in index_confidence_list:
        df_window=df_window.append(df_timestamp_window.iloc[i],ignore_index=True)

    #df_window.mean()series form of all averaged value for the features during the time window
    video_mean_features = df_window.mean().values[3:11]#final features for one switch (291 video features totally)
    video_var_features = df_window.var().values[3:11]
    video_std_features = df_window.std().values[3:11]
    return video_mean_features, video_var_features, video_std_features

In [57]:
# video features in the past timeWindow=3/5/10 seconds
timeWindow = 3
list_mean_features_video = []
list_var_features_video = []
list_std_features_video = []
valid_index=[]

for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))

    print(recording)
    
    # open each of the recording file
    
    with open (os.path.join(recording_path, "video_data.txt")) as video_file:
        video_data = json.load(video_file)["video_data"]
    
        #num_session = len(video_data)
        #????????????????????????????????????????????? something wrong with the video_data for recording 001&002
        
        # count the number of sessions
        list_session = []
        for file in os.listdir(recording_path):
            if str(file)[:7] == "Session":
                list_session.append(str(file))
        
        if list_session==[]:
            continue
        
        
        # open each of the session file
        for session in list_session:
            session_path = os.path.join(recording_path, session)
            
            assert(os.path.exists(session_path))
            
            #get the timestamp which the video started recording
            sessionStartTime = video_data[int(session[-1])]["startTime"]
            sessionStopTime = video_data[int(session[-1])]["stopTime"]
            
            
            if os.path.exists((os.path.join(session_path, "r_video.csv")))==False:
                continue
            df_temp = pd.read_csv(os.path.join(session_path, "r_video.csv"))

            print(session)
  
            for switchingTime in feature_switchingTime:
                if switchingTime < sessionStopTime and switchingTime > sessionStartTime:

                    mean_video_features_switching, var_video_features_switching, std_video_features_switching = extractFeatures(df_temp, switchingTime, sessionStartTime, timeWindow)
                    isnan1 = np.isnan(np.array(mean_video_features_switching.tolist()))
                    isnan2 = np.isnan(np.array(var_video_features_switching.tolist()))
                    isnan3 = np.isnan(np.array(std_video_features_switching.tolist()))
                    p1 = True in isnan1
                    p2 = True in isnan2
                    p3 = True in isnan3
                    if p1==False and p2==False and p3==False:
                        valid_index.append(feature_switchingTime.index(switchingTime))
                        list_mean_features_video.append(mean_video_features_switching.tolist())
                        list_var_features_video.append(var_video_features_switching.tolist())
                        list_std_features_video.append(std_video_features_switching.tolist())
features_mean_video = np.array(list_mean_features_video)
features_var_video = np.array(list_var_features_video)
features_std_video = np.array(list_std_features_video)
print(valid_index)

Recording 000
Session 0000
Session 0002
Session 0003
Session 0004
Session 0005
Session 0006
Session 0007
Session 0008
Session 0009
Session 0010
Session 0011
Recording 001
Session 0000
Session 0001
Session 0002
Session 0003
Session 0004
Session 0005
Session 0006
Session 0007
Session 0008
Session 0009
Session 0010
Session 0011
Session 0012
Session 0013
Session 0014
Session 0015
Session 0016
Session 0017
Session 0018
Session 0019
Session 0020
Session 0021
Session 0022
Recording 002
Session 0000
Session 0001
Session 0002
Recording 003
Session 0000
Recording 004
Session 0000
Session 0001
Session 0002
Recording 005
Session 0000
Session 0001
Recording 006
Session 0000
Session 0001
Session 0002
Session 0003
Session 0004
Session 0005
Session 0006
Session 0007
Session 0008
Recording 007
Recording 008
Session 0000
Session 0001
Session 0002
Session 0003
Session 0004
Session 0005
Session 0006
Session 0007
Session 0008
Session 0009
Session 0010
Session 0011
Session 0012
Session 0013
Session 0014
Ses

In [62]:
features_mean_video = np.array(list_mean_features_video)
features_var_video = np.array(list_var_features_video)
features_std_video = np.array(list_std_features_video)
print(valid_index)

[0, 7, 8, 0, 1, 2, 21, 25, 26, 28, 37, 38, 39, 40, 41, 21, 28, 35, 125, 126, 127, 128, 129, 130, 131, 132, 133, 148, 149, 150, 151, 152, 153, 154, 155, 156, 162, 164, 198, 212, 213, 214, 215, 219, 220, 221, 222, 224, 227, 228, 229, 230, 231, 232, 233, 234, 237, 238, 239, 240, 241, 242, 206, 207, 212, 213, 216, 217, 218, 219, 220, 221, 222, 273, 325, 327, 328, 329, 611, 612, 613, 334, 616, 621, 624, 625, 626, 627, 338, 339, 340, 341, 342, 345, 347, 349, 350, 362, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 338, 339, 340, 341, 342, 349, 350, 352, 353, 354, 355, 356, 357, 366, 371, 372, 373, 374, 375, 376, 338, 339, 340, 341, 342, 423, 424, 425, 426, 427, 428, 430, 431, 432, 433, 434, 435, 437, 438, 439, 441, 444, 445, 446, 447, 448, 426, 432, 433, 436, 437, 438, 439, 440, 441, 444, 445, 446, 447, 448, 430, 432, 433, 435, 436, 437, 438, 439, 440, 441, 442, 502, 503, 504, 505, 506, 507, 509, 512, 513, 515, 516, 517, 521, 523, 524, 525, 526, 502

In [63]:
# There's a shrinkage of the dataset between feature_video(110) and feature_app(128)
# The problems can be:
# 1. There are no Session document in Recording but video_data and application_data show the application switching
# 2. Video_data shows there are n totally sessions but only m(m<n) sessions instead

In [64]:
feature_valid_app=[]
feature_valid_switchingTime=[]
valid_label=[]
for index in valid_index:
    feature_valid_app.append(feature_app[index][0])
    feature_valid_switchingTime.append(feature_switchingTime[index])
    valid_label.append(np_label[index])
feature_valid_app = np.array(feature_valid_app).reshape(len(feature_valid_app),1)
valid_label = np.array(valid_label)
print(feature_valid_app.shape)

(286, 1)


In [65]:
valid_label.shape

(286,)

# Feature of app category:

In [66]:
app

['com.android.settings',
 'com.android.calendar',
 'com.google.android.youtube',
 'com.android.chrome',
 'com.android.email',
 'com.facebook.katana',
 'com.espn.score_center',
 'com.instagram.android',
 'com.whatsapp',
 'de.motain.iliga',
 'com.popularapp.thirtydayfitnesschallenge',
 'com.google.android.apps.maps',
 'com.android.deskclock',
 'com.android.systemui',
 'com.android.incallui',
 'com.google.android.gm',
 'com.bleacherreport.android.teamstream',
 'com.miui.player',
 'com.android.vending',
 'com.fision',
 'com.weather.Weather',
 'com.app.tgtg',
 'com.google.android.apps.translate',
 'com.ubs.swidKXJ.android',
 'com.ubs.swidK2Y.android',
 'com.miui.cleanmaster',
 'com.mi.android.globalFileexplorer',
 'android',
 'com.miui.gallery',
 'ch.publibike.app',
 'com.UCMobile.intl',
 'com.facebook.orca',
 'homedia.sky.sport',
 'com.android.contacts',
 'com.miui.securitycenter',
 'jp.naver.line.android',
 'fm.player',
 'com.airbnb.android',
 'cn.wps.moffice_eng',
 'com.google.android.pa

In [67]:
category = ['financial', 'explorer', 'tools', 'shopping_food_travel',  'sports', 'efficiency', 'entertainment', 'social_media', 'system', 'others']
dict_app = {'financial': ['ro.bcr.georgego','com.eTokenBCR','com.revolut.revolut', 'com.ubs.swidKXJ.android', 'com.ubs.swidK2Y.android',],
                   'explorer': [ 'com.android.chrome','com.android.browser', 'com.android.htmlviewer','com.android.chrome', 'com.UCMobile.intl'],
                   'tools': [ 'com.huawei.search', 'com.android.calendar','com.google.android.apps.maps', 'ch.admin.meteoswiss','com.android.deskclock','com.android.deskclock','com.google.android.apps.maps','com.android.calendar', 'com.weather.Weather'],
                   'shopping_food_travel': [ 'com.wahanda.marketplace', 'com.contextlogic.wish', 'com.fision',  'com.app.tgtg', 'com.airbnb.android'],
                   'sports': [ 'com.espn.score_center', 'de.motain.iliga', 'com.popularapp.thirtydayfitnesschallenge', 'com.bleacherreport.android.teamstream', 'ch.publibike.app', 'homedia.sky.sport'],
                   'efficiency': ['com.google.android.gm', 'com.microsoft.office.word','com.example.android.notepad', 'com.thisclicks.wiw','com.google.android.googlequicksearchbox','com.google.android.gm','com.google.android.gms', 'com.android.email', 'com.google.android.apps.translate', 'cn.wps.moffice_eng'],
                   'entertainment': ['com.apple.android.music','com.google.android.apps.books', 'com.huawei.camera', 'com.google.android.youtube', 'com.adobe.lrmobile','com.huawei.himovie.overseas', 'com.netflix.mediaclient','com.google.android.youtube','com.spotify.music', 'com.android.camera', 'com.miui.player', 'fm.player'],
                   'social_media': ['com.instagram.android', 'com.whatsapp','com.facebook.katana', 'com.android.contacts', 'com.pinterest','com.instagram.android', 'com.whatsapp', 'com.facebook.orca', 'com.facebook.katana', 'com.android.mms','com.android.contacts','jp.naver.line.android',  'com.Slack'],
                   'system': [ 'com.huawei.systemmanager', 'com.ohh2ahh.zuerimensa', 'com.android.incallui','com.android.settings', 'com.huawei.android.internal.app', 'com.android.gallery3d','com.android.vending', 'com.huawei.android.hwouc','com.miui.gallery', 'com.android.settings', 'com.miui.securitycenter', 'com.miui.cleanmaster', 'com.android.providers.downloads.ui', 'com.android.vending',  'com.android.systemui', 'com.android.incallui', 'com.mi.android.globalFileexplorer','android']
                  }

In [68]:
list_feature_app_category = []
for digit in input:
    for key, value in dict_app.items():
        for element in value:
            if element== app[digit]:
                list_feature_app_category.append(category.index(key))
                break
feature_app_category = np.array(list_feature_app_category).reshape(len(list_feature_app_category),1)
feature_app_category.shape

(664, 1)

In [69]:
feature_valid_app_category=[]
for index in valid_index:
    feature_valid_app_category.append(feature_app_category[index][0])
feature_valid_app_category = np.array(feature_valid_app_category).reshape(len(feature_valid_app_category),1)
print(feature_valid_app_category.shape)

(286, 1)


# Label with categories

In [70]:
list_label_category = []
for digit in label:
    for key, value in dict_app.items():
        for element in value:
            if element== app[digit]:
                list_label_category.append(category.index(key))
                break
label_category = np.array(list_label_category).reshape(len(list_label_category),1)
label_valid_category=[]
for index in valid_index:
    label_valid_category.append(label_category[index][0])
label_valid_category = np.array(label_valid_category).reshape(len(label_valid_category),)
print(label_valid_category.shape)

(286,)


# Feature of app frequency

In [71]:
hash_app_frequency = {}
for digit in input:
    hash_app_frequency[digit] = (hash_app_frequency.get(digit, 0) + 1)
for digit in label:
    hash_app_frequency[digit] = (hash_app_frequency.get(digit, 0) + 1)
    
list_app_frequency = []
for digit in input:
    list_app_frequency.append(hash_app_frequency.get(digit, 0))
feature_app_frequency = np.array(list_app_frequency).reshape(len(list_app_frequency),1)
feature_app_frequency.shape

(665, 1)

In [72]:
feature_valid_app_frequency=[]
for index in valid_index:
    feature_valid_app_frequency.append(feature_app_frequency[index][0])
feature_valid_app_frequency = np.array(feature_valid_app_frequency).reshape(len(feature_valid_app_frequency),1)
print(feature_valid_app_frequency.shape)

(286, 1)


# Feature Duration

In [73]:
feature_duration = np.array(duration).reshape(len(duration),1)
feature_valid_duration = []
for index in valid_index:
    feature_valid_duration.append(feature_duration[index][0])
feature_valid_duration = np.array(feature_valid_duration).reshape(len(feature_valid_duration),1)
print(feature_valid_duration.shape)

(286, 1)


# Feature TimeOfTheDay

In [74]:
# switching time:
feature_switchingTime = []
for element in temp_switchingTime:
    feature_switchingTime.append(int(element))

    
# switching timedate
temp = []
for index in range(len(temp_switchingTimeDate)):
    temp.append(temp_switchingTimeDate[index][11:13]+temp_switchingTimeDate[index][14:16]+temp_switchingTimeDate[index][17:19])
feature_switchingTimeDate = []
for element in temp:
    feature_switchingTimeDate.append(int(element))
tt_feature_switchingTimeDate = np.array(feature_switchingTimeDate).reshape(len(feature_switchingTimeDate),1)
feature_valid_switchingTimeDate = []
for index in valid_index:
    feature_valid_switchingTimeDate.append(tt_feature_switchingTimeDate[index][0])
feature_valid_switchingTimeDate = np.array(feature_valid_switchingTimeDate).reshape(len(feature_valid_switchingTimeDate),1)
print(feature_valid_switchingTimeDate.shape)

(286, 1)


# Feature DeviceStatus(3/5/10)

In [75]:
deviceStatus3=[0]*len(feature_switchingTime)
deviceStatus5=[0]*len(feature_switchingTime)
deviceStatus10=[0]*len(feature_switchingTime)
for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))

    if os.path.exists((os.path.join(recording_path, "video_data.txt")))==False:
        continue
    # open each of the recording file
    
    with open (os.path.join(recording_path, "video_data.txt")) as video_file:
        video_data = json.load(video_file)["video_data"]
        #num_session = len(video_data)
        #????????????????????????????????????????????? something wrong with the video_data for recording 001&002
        
            
        with open(os.path.join(recording_path, "device_data.txt")) as device_file:
            device_data = json.load(device_file)["device_data"]

            for d_index in range(len(device_data)):
                for s_index in range(len(feature_switchingTime)):
                    if device_data[d_index]["time"] >= feature_switchingTime[s_index] - 3*1_000_000_000 and device_data[d_index]["time"] <= feature_switchingTime[s_index]:
                        deviceStatus3[s_index]+=1
                    if device_data[d_index]["time"] >= feature_switchingTime[s_index] - 5*1_000_000_000 and device_data[d_index]["time"] <= feature_switchingTime[s_index]:
                        deviceStatus5[s_index]+=1                    
                    if device_data[d_index]["time"] >= feature_switchingTime[s_index] - 10*1_000_000_000 and device_data[d_index]["time"] <= feature_switchingTime[s_index]:
                        deviceStatus10[s_index]+=1            
print(len(deviceStatus3))

665


In [76]:
feature_deviceStatus3 = np.array(deviceStatus3).reshape(len(deviceStatus3),1)
feature_valid_deviceStatus3 = []
for index in valid_index:
    feature_valid_deviceStatus3.append(feature_deviceStatus3[index][0])
feature_valid_deviceStatus3 = np.array(feature_valid_deviceStatus3).reshape(len(feature_valid_deviceStatus3),1)
print(feature_valid_deviceStatus3.shape)

(286, 1)


# Feature TouchEvent(3/5/10)

In [77]:
# touch data: happened touch events in the past 3/5/10 seconds
touchTimes_data3=[0]*len(feature_switchingTime)
touchTimes_data5=[0]*len(feature_switchingTime)
touchTimes_data10=[0]*len(feature_switchingTime)
for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))

    if os.path.exists((os.path.join(recording_path, "video_data.txt")))==False:
        continue
    # open each of the recording file
    
    with open (os.path.join(recording_path, "video_data.txt")) as video_file:
        video_data = json.load(video_file)["video_data"]
        #num_session = len(video_data)
        #????????????????????????????????????????????? something wrong with the video_data for recording 001&002
        

        for file in os.listdir(recording_path):
            if str(file)[:7] == "Session":
                # open each of the session file
                session_path = os.path.join(recording_path, str(file))

                assert(os.path.exists(session_path))

                with open(os.path.join(session_path, "touch_data.txt")) as touch_file:
                    touch_data = json.load(touch_file)["touch_data"]

                    for t_index in range(len(touch_data)):
                        for s_index in range(len(feature_switchingTime)):
                            if touch_data[t_index]["time"] >= feature_switchingTime[s_index] - 3*1_000_000_000 and touch_data[t_index]["time"] <= feature_switchingTime[s_index]:
                                touchTimes_data3[s_index]+=1
                            if touch_data[t_index]["time"] >= feature_switchingTime[s_index] - 5*1_000_000_000 and touch_data[t_index]["time"] <= feature_switchingTime[s_index]:
                                touchTimes_data5[s_index]+=1                    
                            if touch_data[t_index]["time"] >= feature_switchingTime[s_index] - 10*1_000_000_000 and touch_data[t_index]["time"] <= feature_switchingTime[s_index]:
                                touchTimes_data10[s_index]+=1            
print(len(touchTimes_data3))

665


In [78]:
feature_touchTimes3 = np.array(touchTimes_data3).reshape(len(touchTimes_data3),1)
feature_valid_touchTimes3 = []
for index in valid_index:
    feature_valid_touchTimes3.append(feature_touchTimes3[index][0])
feature_valid_touchTimes3 = np.array(feature_valid_touchTimes3).reshape(len(feature_valid_touchTimes3),1)
print(feature_valid_touchTimes3.shape)

(286, 1)


# valid Label

In [79]:
label_app = np.array(label).reshape(len(label),1)
label_valid_app=[]
for index in valid_index:
    label_valid_app.append(label_app[index][0])
label_valid_app = np.array(label_valid_app).reshape(len(label_valid_category),)
print(label_valid_app.shape)

(286,)


# Function

In [80]:
def predict_top_k(X_test, y_test, model, k=3):

    probs = model.predict_proba(X_test)
    labels = model.classes_
    result = 0

    for i in range(probs.shape[0]):
        a = probs[i]
        index = heapq.nlargest(k, range(probs.shape[1]), a.take)
        for element in index:
            if labels[element] == y_test[i]:
                result+=1
    
    return result/y_test.shape[0]

def kNN_training (feature, label):
    valid_X = scaler.fit_transform(feature)
    valid_y = label
    X_train, X_test, y_train, y_test = train_test_split(valid_X, valid_y, random_state=0)
    model = KNeighborsClassifier(n_neighbors=7, weights='uniform').fit(X_train,y_train)
    accuracy = model.score(X_test, y_test)
    top_accuracy4 = predict_top_k(X_test, y_test, model, k=4)
    top_accuracy8 = predict_top_k(X_test, y_test, model, k=8)
    return accuracy, top_accuracy4, top_accuracy8

def rf_training (feature, label):
    valid_X = scaler.fit_transform(feature)
    valid_y = label
    X_train, X_test, y_train, y_test = train_test_split(valid_X, valid_y, random_state=0)
    model = RandomForestClassifier(n_estimators=50).fit(X_train,y_train)
    accuracy = model.score(X_test, y_test)
    top_accuracy4 = predict_top_k(X_test, y_test, model, k=4)
    top_accuracy8 = predict_top_k(X_test, y_test, model, k=8)
    return accuracy, top_accuracy4, top_accuracy8

def Ada_training (feature, label):
    valid_X = scaler.fit_transform(feature)
    valid_y = label
    X_train, X_test, y_train, y_test = train_test_split(valid_X, valid_y, random_state=0)
    model = AdaBoostClassifier(n_estimators=100, random_state=0).fit(X_train,y_train)
    accuracy = model.score(X_test, y_test)
    top_accuracy4 = predict_top_k(X_test, y_test, model, k=4)
    top_accuracy8 = predict_top_k(X_test, y_test, model, k=8)
    return accuracy, top_accuracy4, top_accuracy8

def dummy_training(feature, label):
    dummy = DummyClassifier(strategy="most_frequent")
    dummy = dummy.fit(feature, label)
    accuracy = dummy.score(feature, label)
    return accuracy

def calculateMeanRF(feature, label):
    accuracy=[]
    top4=[]
    top8=[]
    for i in range(5):
        accuracy_rf, top_accuracy_rf4, top_accuracy_rf8= rf_training (feature, label)
        accuracy.append(accuracy_rf)
        top4.append(top_accuracy_rf4)
        top8.append(top_accuracy_rf8)
    return sum(accuracy)/len(accuracy), sum(top4)/len(top4), sum(top8)/len(top8)

def calculateMeanKNN(feature, label):
    accuracy=[]
    top4=[]
    top8=[]
    for i in range(5):
        accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8= kNN_training (feature, label)
        accuracy.append(accuracy_kNN)
        top4.append(top_accuracy_kNN4)
        top8.append(top_accuracy_kNN8)
    return sum(accuracy)/len(accuracy), sum(top4)/len(top4), sum(top8)/len(top8)

def calculateMeanAda(feature, label):
    accuracy=[]
    top4=[]
    top8=[]
    for i in range(5):
        accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8= Ada_training (feature, label)
        accuracy.append(accuracy_kNN)
        top4.append(top_accuracy_kNN4)
        top8.append(top_accuracy_kNN8)
    return sum(accuracy)/len(accuracy), sum(top4)/len(top4), sum(top8)/len(top8)

# Feature Summary

In [81]:
print(feature_valid_app.shape)
print(feature_valid_duration.shape)
print(feature_valid_switchingTimeDate.shape)
print(feature_valid_deviceStatus3.shape)
print(feature_valid_touchTimes3.shape)
print(feature_valid_app_category.shape)
print(feature_valid_app_frequency.shape)
print(features_mean_video.shape)
print(features_var_video.shape)
print(features_std_video.shape)
print(label_valid_category.shape)
print(label_valid_app.shape)

(286, 1)
(286, 1)
(286, 1)
(286, 1)
(286, 1)
(286, 1)
(286, 1)
(286, 8)
(286, 8)
(286, 8)
(286,)
(286,)


In [82]:
allFeatures = np.concatenate([feature_valid_app,
                              feature_valid_duration,
                              feature_valid_switchingTimeDate,
                              feature_valid_deviceStatus3,
                              feature_valid_touchTimes3,
                              feature_valid_app_category, 
                              feature_valid_app_frequency, 
                              features_mean_video, 
                              features_var_video,
                              features_std_video],
                        axis=1)
print(allFeatures.shape)

(286, 31)


In [83]:
isnan = np.isnan(features_std_video) # 判断每个元素是不是nan,返回[False,False,False,False,True]

print(True in isnan) # 判断isnan中是否包含True, 返回 True

False


# Feature Selection

In [84]:
#L1-based feature selection for all 288+1 features
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

linearsvc = LinearSVC(C=0.01, penalty='l1', dual=False).fit(allFeatures, label_valid_app)
model = SelectFromModel(linearsvc, prefit=True)
X_new_l1 = model.transform(allFeatures)
X_new_l1.shape

D:\Softwares\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


(286, 5)

In [85]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=100).fit(allFeatures, label_valid_app)
model = SelectFromModel(clf, prefit=True)
X_new_tree = model.transform(allFeatures)
X_new_tree.shape

(286, 6)

# Model Performance

In [86]:
accuracy_rf, top_accuracy_rf4, top_accuracy_rf8 = calculateMeanRF(X_new_tree, label_valid_app)
print(accuracy_rf, top_accuracy_rf4, top_accuracy_rf8)

0.6027777777777779 0.7527777777777778 0.8083333333333333


In [87]:
accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8 = calculateMeanKNN(X_new_tree, label_valid_app)
print(accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8)

0.3194444444444444 0.6944444444444444 0.8055555555555556


In [88]:
accuracy_ada, top_accuracy_ada4, top_accuracy_ada8 = calculateMeanAda(X_new_tree, label_valid_app)
print(accuracy_ada, top_accuracy_ada4, top_accuracy_ada8)

0.19444444444444445 0.5972222222222222 0.7083333333333334


# =======================================

In [89]:
accuracy_dummy = dummy_training(feature_valid_app, label_valid_app)
print(accuracy_dummy)

0.1993006993006993


# =====================================

In [ ]:
# 1. probabilistic model                            check
# 2. columns manuelly select                        check
# 3. mean, variance, standard deviation             check
# 4. weights of the frequency                       check
# 5. categorical app                                check